In [1]:
import json
from pathlib import Path

data = json.load(open(f"{Path.home()}/data/vivid_video_instruct_128k_all.json"))

In [6]:
longs = [x for x in data if len(x["conversations"]) > 2 * 3]
shorts = [x for x in data if len(x["conversations"]) <= 2 * 3]

In [33]:
superlongs = []

for i in range(0, 10000):
  # total word count incrementally of all conversations, i.e. 1st, 1st+2nd, 1st+2nd+3rd, etc.

  total_word_count = sum(
    [
      sum([len(x["value"].split(" ")) for x in data[i]["conversations"][:j]])
      for j in range(1, len(data[i]["conversations"]) + 1)
    ]
  )
  if total_word_count > 5000:
    superlongs.append((i, len(data[i]["conversations"]), data[i]["conversations"]))

In [5]:
sorted_data = sorted(
  data,
  key=lambda x: sum(
    [
      sum([len(x["value"].split(" ")) for x in x["conversations"][:j]])
      for j in range(1, len(x["conversations"]) + 1)
    ]
  ),
  reverse=True,
)

In [6]:
len(sorted_data)

128880

In [7]:
sorted_data[0]

{'id': 'egu0jca7G8o.9',
 'video': 'shard_23/egu0jca7G8o.9.mp4',
 'conversations': [{'from': 'human',
   'value': '<video>\nMàn hình đầu tiên của video là gì?'},
  {'from': 'gpt', 'value': 'Cảnh một người đàn ông đang bơi dưới nước.'},
  {'from': 'human',
   'value': 'Sau cảnh người đàn ông bơi dưới nước, cảnh tiếp theo là gì?'},
  {'from': 'gpt',
   'value': 'Cảnh một người đàn ông đang lái thuyền trên biển.'},
  {'from': 'human', 'value': 'Trên thuyền có những ai?'},
  {'from': 'gpt', 'value': 'Có nhiều người trên thuyền, họ đang mặc áo phao.'},
  {'from': 'human', 'value': 'Người phụ nữ trên thuyền đang làm gì?'},
  {'from': 'gpt', 'value': 'Đang ngồi trên thuyền và sử dụng điện thoại.'},
  {'from': 'human',
   'value': 'Cảnh tiếp theo sau người phụ nữ sử dụng điện thoại là gì?'},
  {'from': 'gpt', 'value': 'Cảnh một chiếc thuyền đang chạy trên biển.'},
  {'from': 'human',
   'value': 'Cảnh tiếp theo sau chiếc thuyền chạy trên biển là gì?'},
  {'from': 'gpt',
   'value': 'Cảnh một nh

In [34]:
len(superlongs)

1

In [35]:
superlongs

[(8471,
  20,
  [{'from': 'human', 'value': 'Video này nói về chủ đề gì?\n<video>'},
   {'from': 'gpt',
    'value': 'Video này thảo luận về lý thuyết rằng các kim tự tháp ở Giza được xây dựng như một nguồn năng lượng khổng lồ, sử dụng các thuộc tính của vật liệu chúng được xây dựng để tạo ra và truyền tải năng lượng.'},
   {'from': 'human', 'value': 'Kim tự tháp Giza được căn chỉnh với cái gì?'},
   {'from': 'gpt',
    'value': 'Video bắt đầu bằng việc nêu bật sự căn chỉnh chính xác của ba kim tự tháp với vành đai của chòm sao Orion, gợi ý về mối liên hệ giữa các kim tự tháp và các thiên thể.'},
   {'from': 'human',
    'value': 'Tại sao đá vôi trắng Tura lại quan trọng trong việc xây dựng kim tự tháp?'},
   {'from': 'gpt',
    'value': 'Đá vôi trắng Tura được sử dụng cho lớp vỏ ngoài của các kim tự tháp, đóng vai trò như một chất cách điện, ngăn ngừa sự mất năng lượng từ cấu trúc.'},
   {'from': 'human',
    'value': 'Cấu trúc bên trong của kim tự tháp bao gồm những vật liệu gì?'},
 

In [110]:
# new_data = []

# for d in data:
#   d_copy = d.copy()
#   new_conv = []
#   for i in range(0, len(d["conversations"]), 2):
#     conv = d["conversations"][i : i + 2]
#     conv[0]["value"] = conv[0]["value"].strip()
#     conv[1]["value"] = conv[1]["value"].strip()
#     if len(conv[1]["value"].split(" ")) > 10:
#       new_conv.extend(conv)
#   if len(new_conv) > 0:
#     d_copy["conversations"] = new_conv[:6]
#     new_data.append(d_copy)

In [2]:
new_data = []
superlongs = []

for d in data:
  d_copy = d.copy()
  new_conv = []
  for i in range(0, len(d["conversations"]), 2):
    conv = d["conversations"][i : i + 2]
    conv[0]["value"] = conv[0]["value"].strip()
    conv[1]["value"] = conv[1]["value"].strip()
    if (
      len(conv[0]["value"].split(" ")) > 2
      and len(conv[0]["value"].split(" ")) < 60
      and len(conv[1]["value"].split(" ")) > 20
      and len(conv[1]["value"].split(" ")) < 300
    ):
      new_conv.extend(conv)
  if len(new_conv) > 0:
    d_copy["conversations"] = []
    total_word_count = 0
    for i in range(0, len(new_conv), 2):
      try:
        total_word_count += total_word_count + len(new_conv[i]["value"].split(" "))
        if total_word_count < 2000:
          d_copy["conversations"].extend(new_conv[i : i + 2])
        else:
          superlongs.append(d)
          break
        total_word_count += len(new_conv[i + 1]["value"].split(" "))
      except:
        pass

    if "<video>" not in d_copy["conversations"][0]["value"]:
      d_copy["conversations"][0]["value"] = (
        "<video>\n" + d_copy["conversations"][0]["value"]
      )

    if len(d_copy["conversations"]) > 0:
      new_data.append(d_copy)

In [3]:
len(new_data)

92218

In [4]:
new_data[-1]

{'id': 'zU633ZJUB4s.6',
 'video': 'shard_69/zU633ZJUB4s.6.mp4',
 'conversations': [{'from': 'human',
   'value': 'Diễn đạt chính xác nội dung video, tập trung vào cốt truyện và hình ảnh quan trọng.\n<video>'},
  {'from': 'gpt',
   'value': 'Video bắt đầu với một bảng gồm hai cột. Cột bên trái có chữ "đồng hồ đó là một chút..." và hai dòng bên dưới là "Nhanh" và "chậm". Cột bên phải có chữ "ese reloj está un poco..." và hai dòng bên dưới là "adelantado" và "atrasado". Bên cạnh bảng là hình ảnh của bốn lá cỏ bốn lá màu xanh lá cây. Phía dưới bảng là hình ảnh của những chấm tròn màu vàng nhạt. Sau đó, video chuyển sang hình ảnh nền mờ với những vòng tròn màu vàng, cam và đỏ. Cuối cùng, video hiển thị dòng chữ "Muchas gracias!!!" màu tím trên nền mờ.'}]}

In [5]:
sorted_new_data = sorted(
  new_data,
  key=lambda x: sum(
    [
      sum([len(x["value"].split(" ")) for x in x["conversations"][:j]])
      for j in range(1, len(x["conversations"]) + 1)
    ]
  ),
  reverse=True,
)

In [6]:
sorted_new_data[0]

{'id': '1uzz3-v_OtE.5',
 'video': 'shard_47/1uzz3-v_OtE.5.mp4',
 'conversations': [{'from': 'human',
   'value': '<video>\nNội dung của video là gì?'},
  {'from': 'gpt',
   'value': 'Video bắt đầu với hình ảnh một nhân vật nữ trong game đang đứng trước một cái bàn, sau đó chuyển sang hình ảnh một thành phố với một đài phun nước ở giữa. Tiếp theo là hình ảnh một nhân vật nữ trong game đang đi trên một con đường, rồi đến hình ảnh nhân vật nữ đó đang chiến đấu với một đàn chim. Cuối cùng, video còn hiển thị các hình ảnh khác như một ứng dụng game trên điện thoại, một người đàn ông ném vật thể vào thùng rác, một túi rác có vật thể màu vàng bên trong, một nhân vật nữ trong game với chiếc mũ hề, một bức tranh biếm họa, hai phụ nữ trẻ nói chuyện trước máy quay, và một meme với một người đàn ông đầu trọc mặc đồng phục cảnh sát.'},
  {'from': 'human', 'value': 'Trong video, có những đối tượng chính nào?'},
  {'from': 'gpt',
   'value': 'Các đối tượng chính trong video bao gồm một nhân vật nữ tr

In [63]:
# new_data = [
#   {**x, "path": ""}
#   for x in sorted_new_data
#   if len(x["conversations"][1]["value"].split(" ")) > 10
# ]

In [7]:
new_data = sorted(new_data, key=lambda x: int(x["video"].split("/")[0].split("_")[-1]))

In [8]:
len(new_data)

92218

In [9]:
new_data = [{**d, "path": "Vividbot/vivid-video-instruct/videos"} for d in new_data]

In [10]:
new_data[-10]

{'id': 'zD5rDjh2whM.4',
 'video': 'shard_129/zD5rDjh2whM.4.mp4',
 'conversations': [{'from': 'human',
   'value': 'Trình bày chi tiết về khung cảnh, nhân vật và sự kiện diễn ra trong video.\n<video>'},
  {'from': 'gpt',
   'value': 'Video giới thiệu về công dụng của lá lốt đối với làn da của chị em phụ nữ. Lá lốt có chứa nhiều chất xơ, tinh dầu và phenol, đây đều là những dưỡng chất tốt cho làn da. Lá lốt được nhiều chị em phụ nữ sử dụng trong công thức làm đẹp của riêng mình.'}],
 'path': 'Vividbot/vivid-video-instruct/videos'}

In [11]:
delist_ids = ["jdQwgqP5WAY.7", "jjAwFs3Yx2o.3", "jjAwFs3Yx2o.7"]

new_data = [x for x in new_data if x["id"] not in delist_ids]

In [ ]:
for d in new_data:
  if len(d["conversations"][1]["value"].split(" ")) <= 30:
    print(d)

In [60]:
custom_data = [
  d
  for d in new_data
  if d["video"].split("/")[0] in [f"shard_{i}" for i in range(110, 120)]
]

In [65]:
custom_data[1202]

{'id': 'UI4GFv7Y4XA.10',
 'video': 'shard_111/UI4GFv7Y4XA.10.mp4',
 'conversations': [{'from': 'human',
   'value': 'Viết một bản mô tả toàn diện về video, nắm bắt được điểm chính và những khoảnh khắc đáng nhớ.\n<video>'},
  {'from': 'gpt',
   'value': 'Video này giới thiệu một cô bé đang chơi với nhiều loại động vật con trong vườn. Cô bé lần lượt chỉ vào các chuồng và nói tên các loại động vật con như thỏ con, chó con, heo con, gà con, mèo con. Sau đó, cô bé chơi đùa với những con chó con và gà con. Cuối cùng, cô bé cùng hai người bạn của mình đứng trước camera và mỗi người cầm một bình sữa.'}],
 'path': 'Vividbot/vivid-video-instruct/videos'}

In [12]:
with open(f"{Path.home()}/data/vivid_video_instruct.json", "w") as f:
  json.dump(new_data, f, indent=2, ensure_ascii=False)

In [13]:
len(new_data)

92213

In [14]:
new_data[0]

{'id': '01gE8ErrcJ4.0',
 'video': 'shard_0/01gE8ErrcJ4.0.mp4',
 'conversations': [{'from': 'human',
   'value': '<video>\nSự khác biệt chính giữa hình ảnh phổi đầu và hình ảnh phổi tiếp theo là gì?'},
  {'from': 'gpt',
   'value': 'Hình ảnh phổi đầu được mô tả với nhiều màu sắc khác nhau, trong khi hình ảnh phổi tiếp theo mô tả cấu trúc của phổi với nhiều túi khí nhỏ.'}],
 'path': 'Vividbot/vivid-video-instruct/videos'}

In [18]:
item = filter(lambda x: x["video"] == "shard_60/zCzlzDW6vHE.4.mp4", new_data)

In [19]:
next(item)

{'id': 'zCzlzDW6vHE.4',
 'video': 'shard_60/zCzlzDW6vHE.4.mp4',
 'conversations': [{'from': 'human',
   'value': '<video>\nCó gì phía sau người đàn ông mặc áo choàng màu vàng ?'},
  {'from': 'gpt',
   'value': 'Phía sau người đàn ông mặc áo choàng màu vàng là một thành phố cổ kính với những tòa nhà cao tầng.'}],
 'path': 'Vividbot/vivid-video-instruct/videos'}

In [66]:
sorted_new_data = sorted(
  new_data,
  key=lambda x: sum(
    [
      sum([len(x["value"].split(" ")) for x in x["conversations"][:j]])
      for j in range(1, len(x["conversations"]) + 1)
    ]
  ),
  reverse=True,
)

In [74]:
i = 1

total_word_count = sum(
  [
    sum([len(x["value"].split(" ")) for x in sorted_new_data[i]["conversations"][:j]])
    for j in range(1, len(sorted_new_data[i]["conversations"]) + 1)
  ]
)
total_word_count, sorted_new_data[i]

(975,
 {'id': 'KRsrPnVGVkE.5',
  'video': 'shard_59/KRsrPnVGVkE.5.mp4',
  'conversations': [{'from': 'human',
    'value': '<video>\nHình ảnh minh họa cho podcast này là gì?'},
   {'from': 'gpt',
    'value': 'Hình ảnh là một căn phòng với một chiếc ghế sofa màu xanh lá cây có đệm sọc trắng và xanh lá cây, trên bàn cạnh ghế sofa có một chậu cây nhỏ, bên cạnh bàn là một chiếc bàn nhỏ khác có một chậu cây nhỏ khác, ở góc trên bên phải của hình ảnh là một chiếc điện thoại thông minh màu xanh lá cây với một biểu tượng phát màu trắng, trên màn hình điện thoại là một hình ảnh minh họa cho một podcast với một cô gái đang đọc sách, ở góc trên bên trái của hình ảnh là một dòng chữ màu xanh dương "Tối giản" và một dòng chữ màu xanh dương "Tối ưu" ở bên dưới, ở giữa hình ảnh là một dòng chữ màu xanh dương "Lối sống cuộc đời" với các chữ cái được trang trí bằng các hình ngôi sao màu trắng, ở góc dưới bên phải của hình ảnh là một dòng chữ màu xanh dương "RADIOCHUALANHTRAITIM".'},
   {'from': 'human

In [1]:
import json
from pathlib import Path

d2 = json.load(open(f"{Path.home()}/data/vivid_video_instruct.json"))

In [2]:
len(d2)

97448

In [4]:
d2[-1]

{'id': 'z3X29hqSTGQ.9',
 'video': 'shard_99/z3X29hqSTGQ.9.mp4',
 'conversations': [{'from': 'human',
   'value': 'Khi nào vụ tấn công tượng đài xảy ra?'},
  {'from': 'gpt',
   'value': 'Khoảng 3 giờ sáng ngày 10 tháng 6 theo giờ địa phương'}],
 'path': 'Vividbot/vivid-video-instruct/videos'}